# Vectorisation avec Python

## Opérations sur des vecteurs

Python fournit heureusement des outils pour effectuer les opérations sur les vecteurs. Parmi les librairies utiles, citons *math*, *Numpy*, *Scipy* et *Scikit-Learn*.

Avant de les charger en fonction de nos besoins, reprenons quelques vecteurs vus dans le précédent calepin :

$$
\vec{A} = \begin{pmatrix}
    3  \\
    12 \\
    9  \\
    0
\end{pmatrix}
\hspace{2em}
\vec{B} = \begin{pmatrix}
    7  \\
    32 \\
    10
\end{pmatrix}
\hspace{2em}
\vec{C} = \begin{pmatrix}
    11 \\
    4  \\
    8
\end{pmatrix}
$$

In [1]:
A = [3, 12, 9, 0]
B = [7, 32, 10]
C = [11, 4, 8]

### Le produit scalaire de deux vecteurs

La méthode `.dot()` permet d’obtenir le produit de deux vecteurs :

In [3]:
import numpy as np

np.dot(A, A) ** .5

15.297058540778355

### La norme d’un vecteur

Nous avions établi précédemment la norme du vecteur $\vec{A}$ approximativement à $15,2971$. Vérifions notre calcul avec la fonction d’algèbre linéaire `norm()` :

In [4]:
np.linalg.norm(A)

15.297058540778355

### La distance entre deux vecteurs

La distance euclidienne entre deux vecteurs a cet avantage qu’elle peut se trouver dans un espace de n’importe quel dimension. L’opération revient à soustraire deux vecteurs puis à obtenir la norme de ce nouveau vecteur :

In [5]:
np.linalg.norm(np.array(C) - np.array(B))

28.35489375751565

**Remarque :** notons que nous avons dû convertir explicitement nos variables `B` et `C`, de type `list` en tableaux *Numpy*. À titre anecdotique, le module `math` se passe de la conversion :

In [6]:
from math import dist

dist(B, C)

28.35489375751565

## Métriques d’évaluation

### La similarité cosinus

Avec *Numpy*, il est nécessaire de déplier la formule :

In [7]:
cos_BC = np.dot(B, C) / np.dot(np.linalg.norm(B), np.linalg.norm(C))

print(f"La similarité cosinus des vecteurs B et C est évaluée à {cos_BC}.")

La similarité cosinus des vecteurs B et C est évaluée à 0.5869455647701575.


Une fonction adéquate existe dans la librairie *Scikit-Learn*, `cosine_similarity`. Il convient toutefois de lui transmettre une matrice aux bonnes dimensions (nombre de vecteurs, nombre d’attributs). Dans notre exemple, la dimension serait (1, 3) parce que nous lui envoyons un vecteur constitué de trois composantes :

In [8]:
from sklearn.metrics.pairwise import cosine_similarity

cos_BC = cosine_similarity(
    np.array(B).reshape(1, -1),
    np.array(C).reshape(1, -1)
)

print(f"La similarité cosinus des vecteurs B et C est évaluée à {cos_BC.ravel().tolist()[0]}.")

La similarité cosinus des vecteurs B et C est évaluée à 0.5869455647701576.


### L’indice de Jaccard

Une fois encore, *Scikit-Learn* permet d’obtenir directement la métrique en comparant deux vecteurs :

In [9]:
from sklearn.metrics import jaccard_score

A = [0, 0, 1, 1, 1]
B = [1, 0, 1, 0, 0]

print(f"Les vecteurs A et B sont similaires à {jaccard_score(A, B):.2%}")

Les vecteurs A et B sont similaires à 25.00%


## Vectoriser un texte

Considérons un corpus restreint de trois phrases :

In [10]:
corpus = [
    "Le petit chat boit du lait.",
    "Le petit chien boit de l’eau.",
    "La vache boit de l’eau mais ne boit pas de lait."
]

### L’approche fréquentielle

#### Obtenir une matrice d’occurrences

Le module *Scikit-Learn* dispose d’outils pour l’extraction de caractéristiques dans un texte :

In [11]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)

Le résultat produit un vocabulaire, que l’on peut récupérer avec la méthode `.get_feature_names_out()` :

In [13]:
vectorizer.get_feature_names_out()

array(['boit', 'chat', 'chien', 'de', 'du', 'eau', 'la', 'lait', 'le',
       'mais', 'ne', 'pas', 'petit', 'vache'], dtype=object)

La méthode `.toarray()` quant à elle permet de représenter la matrice creuse du corpus :

In [14]:
X.toarray()

array([[1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0],
       [1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0],
       [2, 0, 0, 2, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1]])

Associée à *Pandas*, il est possible de révéler le nom des colonnes afin de nous apprendre que dans la dernière phrase le mot *de* apparaît deux fois :

In [15]:
import pandas as pd

df = pd.DataFrame(
    data=X.toarray(),
    columns=vectorizer.get_feature_names_out()
)

print(df)

   boit  chat  chien  de  du  eau  la  lait  le  mais  ne  pas  petit  vache
0     1     1      0   0   1    0   0     1   1     0   0    0      1      0
1     1     0      1   1   0    1   0     0   1     0   0    0      1      0
2     2     0      0   2   0    1   1     1   0     1   1    1      0      1


Il est également possible de transmettre directement un vocabulaire au constructeur plutôt que de lui laisser la charge de le construire :

In [16]:
vocabulary = ['boit', 'chat', 'chien', 'eau', 'lait', 'petit', 'vache']
vectorizer = CountVectorizer(vocabulary=vocabulary)
X = vectorizer.fit_transform(corpus)

X.toarray()

array([[1, 1, 0, 0, 1, 1, 0],
       [1, 0, 1, 1, 0, 1, 0],
       [2, 0, 0, 1, 1, 0, 1]])

#### Calculer une matrice de cooccurrences

Une matrice de cooccurrences relève la fréquence d’apparition d’un terme en compagnie de chacun des autres termes du corpus. Elle se calcule simplement en algèbre linéaire grâce au produit de la transposée de la matrice d’occurrences avec elle-même :

In [17]:
m = X.toarray()
coocc = m.T.dot(m)
np.fill_diagonal(coocc, 0)

pd.DataFrame(
    data=coocc,
    columns=vectorizer.get_feature_names_out(),
    index=vectorizer.get_feature_names_out())

,boit,chat,chien,eau,lait,petit,vache
boit,0,1,1,3,3,2,2
chat,1,0,0,0,1,1,0
chien,1,0,0,1,0,1,0
eau,3,0,1,0,1,1,1
lait,3,1,0,1,0,1,1
petit,2,1,1,1,1,0,0
vache,2,0,0,1,1,0,0


### L’encodage *one-hot*

Parfois, l’objectif est simplement de repérer, pour un document, quels mots du vocabulaire apparaissent, sans compter forcément sa fréquence. Ce genre d’opération s’obtient à partir de la matrice d’occurrences par un encodage *one-hot*, ou encodage 1 parmi *n*, en fixant à 1 tous les éléments qui ne sont pas égaux à 0 :

In [18]:
list(map(lambda row: [ 0 if e == 0 else 1 for e in row ], X.toarray()))

[[1, 1, 0, 0, 1, 1, 0], [1, 0, 1, 1, 0, 1, 0], [1, 0, 0, 1, 1, 0, 1]]

### La pondération TF-IDF

#### À partir d’une matrice d’occurrences

À partir d’une matrice d’occurrences, obtenue par exemple avec le transformateur `CountVectorizer`, il est possible de calculer une matrice TF (*term frequency*) ou TF-IDF (*term frequency-inverse document frequency*). C’est le rôle du transformateur `TfidfTransformer` :

In [19]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer()
result = tfidf.fit_transform(X)

Le résultat est une matrice creuse dont il est possible de n’afficher que les éléments différents de 0 :

In [20]:
print(result)

  (0, 5)	0.4804583972923858
  (0, 4)	0.4804583972923858
  (0, 1)	0.6317450542765208
  (0, 0)	0.3731188059313277
  (1, 5)	0.4804583972923858
  (1, 3)	0.4804583972923858
  (1, 2)	0.6317450542765208
  (1, 0)	0.3731188059313277
  (2, 6)	0.5305873490316616
  (2, 4)	0.40352535506797127
  (2, 3)	0.40352535506797127
  (2, 0)	0.6267468712982053


Chaque ligne donne la mesure TF-IDF d’un mot dans un document. Prenons la ligne :

```txt
(1, 5)	0.4804583972923858
```

Il faut comprendre ici que dans le document n°1, la mesure TF-IDF du terme n°5 est de 0,48046. Autrement dit, l’importance du terme *petit* est évaluée à 0,48046 dans la deuxième phrase de notre corpus.

Il est à noter que le transformateur ne garde pas trace des caractéristiques apprises, juste de leurs indices :

In [21]:
tfidf.get_feature_names_out()

array(['x0', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6'], dtype=object)

Pour les retrouver, il faut interroger la matrice d’occurrences :

In [22]:
vectorizer.get_feature_names_out()

array(['boit', 'chat', 'chien', 'eau', 'lait', 'petit', 'vache'],
      dtype=object)

#### Sans matrice d’occurrences

Le même résultat s’obtient directement depuis le corpus avec le transformateur `TfidfVectorizer` :

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(vocabulary=vocabulary)
tfidf = vectorizer.fit_transform(corpus)

print(tfidf)

  (0, 5)	0.4804583972923858
  (0, 4)	0.4804583972923858
  (0, 1)	0.6317450542765208
  (0, 0)	0.3731188059313277
  (1, 5)	0.4804583972923858
  (1, 3)	0.4804583972923858
  (1, 2)	0.6317450542765208
  (1, 0)	0.3731188059313277
  (2, 6)	0.5305873490316616
  (2, 4)	0.40352535506797127
  (2, 3)	0.40352535506797127
  (2, 0)	0.6267468712982053


#### Visualisation sous forme tabulaire

La visualisation que l’on attend d’une matrice prend souvent la forme d’un tableau :

|Document|boit|chat|chien|eau|lait|petit|vache|
|:-:|-|-|-|-|-|-|-|
|0|0.373119|0.631745|0|0|0.480458|0.480458|0|
|1|0.373119|0|0.631745|0.480458|0|0.480458|0|
|2|0.626747|0|0|0.403525|0.403525|0|0.530587|

Notre corpus étant restreint et la plupart des mots du vocabulaire étant présents dans chaque document qui le compose, cette représentation est plutôt explicite. Il n’en sera **jamais** de même avec des données réelles. La matrice TF-IDF est une matrice creuse :

In [ ]:
type(tfidf)

Si *Scikit-Learn* choisit ce mode de représentation, c’est parce qu’une matrice creuse étant constituée majoritairement de 0, elle ne prend que peu de place en mémoire. Si toutefois nous souhaitons obtenir une visualisation tabulaire, il faut au préalable la convertir en une matrice dense :

In [ ]:
tfidf_dense = tfidf.todense()

Puis la transmettre à *Pandas* en important les en-têtes de colonnes :

In [ ]:
pd.DataFrame(data=tfidf_dense, columns=vectorizer.get_feature_names_out())

### L’information mutuelle ponctuelle

Mieux connu sous l’acronyme PMI de l’anglais *Pointwise mutual information*, le concept a été présenté en 1961 par l’italien Roberto Mario Fano dans son ouvrage *Transmission of Information: A Statistical Theory of Communications* publié aux éditions du MIT. Il peut être décrit comme une mesure de la fréquence de deux événements par rapport à ce que l’on attendrait s’ils étaient indépendants, sachant que la probabilité de la survenue de deux événements indépendants est simplement le produit de leurs probabilités respectives. La formule vaut ainsi :

$$
\text{PMI}(x,y) = \log_2 \frac{P(x, y)}{P(x) \cdot P(y)}
$$

Le résultat allant de $-\infty$ à $+\infty$, on utilise plus volontiers, en traitement automatique du langage naturel, la variante strictement positive, la PPMI. La technique consiste simplement à fixer à 0 tout résultat inférieur à 0.

#### Une matrice d’occurrences des *n*-grammes

La seule matrice d’occurrences des unigrammes ne nous permettrait de calculer, dans la formule, que les termes $P(x)$ ou $P(y)$. Par exemple, pour le mot *boit* qui apparaît dans tout le corpus 4 fois sur un total de 13 occurrences pour tous les mots du vocabulaire, sa probabilité serait de 0.30770 :

In [ ]:
unigrams = X.toarray()
p_unigrams = unigrams.sum(axis=0) / unigrams.sum().sum()

df_unigrams = pd.DataFrame(
    data=p_unigrams,
    index=vectorizer.get_feature_names_out(),
    columns=['p'])

df_unigrams

Pour obtenir la même matrice pour les bigrammes du corpus, il est possible d’appeler une autre instance de la classe `CountVectorizer` en lui adjoignant le paramètre `ngram_range` :

In [ ]:
vect_bigrams = CountVectorizer(analyzer='word', ngram_range=(2, 2))
X2 = vect_bigrams.fit_transform(corpus)

bigrams = X2.toarray()
p_bigrams = bigrams.sum(axis=0) / bigrams.sum().sum()

df_bigrams = pd.DataFrame(
    data=p_bigrams,
    index=vect_bigrams.get_feature_names_out(),
    columns=['p'])

df_bigrams

#### Une matrice PPMI

Intéressons-nous aux termes *petit* et *chat* afin de calculer leur PMI :

$$
\begin{align}
    \text{PMI}(\text{petit},\text{chat}) &= \log_2 \frac{P(\text{petit}, \text{chat})}{P(\text{petit}) \cdot P(\text{chat})} \\
    &= \log_2 \frac{0.05}{0.153846 \times 0.076923} \\
    &= \log_2 4.225008 \\
    &= 2.07895
\end{align}
$$

Fusionnons tout d’abord les listes de probabilités dans un dictionnaire où la clé sera le terme, un unigramme ou un bigramme, et la valeur la probabilité :

In [ ]:
probabilities = dict()
probabilities.update(df_unigrams.p.to_dict())
probabilities.update(df_bigrams.p.to_dict())

Définissons maintenant une fonction qui calcule la PMI de deux termes à partir d’un dictionnaire des probabilités :

In [ ]:
def PPMI(*, word, context, probabilities):
    """Positive Pointwise Mutual Information.
    
    Keyword arguments:
    word -- a word (w)
    context -- the context of the word (c)
    probabilities -- probabilities for P(w), P(c) and P(w, c)
    """
    wc = probabilities.get(f"{word} {context}", 0)
    cw = probabilities.get(f"{context} {word}", 0)

    expected = probabilities.get(word, 0) * probabilities.get(context, 0)

    # division by zero is undefined
    if expected == 0:
        score = 0.0
    else:
        # silence warnings about log(0)
        with np.errstate(divide='ignore'):
            score = np.log2( (wc + cw) / expected )
        # log(0) = 0
        score = 0.0 if np.isinf(score) or score < 0 else score

    return score

Vérifions que la PPMI de *petit* et *chat* correspond à ce que nous avons calculé à la main :

In [ ]:
PPMI(word='petit', context='chat', probabilities=probabilities)

Pour construire la matrice PPMI, nous avons besoin d’une matrice carrée et symétrique des termes du vocabulaire :

In [ ]:
vectors = np.array([
    [
        PPMI(word=w1, context=w2, probabilities=probabilities)
        for w2 in vectorizer.get_feature_names_out()
    ]
    for w1 in vectorizer.get_feature_names_out()
])

pd.DataFrame(
    data=vectors,
    index=vectorizer.get_feature_names_out(),
    columns=vectorizer.get_feature_names_out())

#### Retrouver des vecteurs similaires

Concept-clé du TAL, la PPMI constitue un excellent moyen pour obtenir des vecteurs de mots. Il est ensuite facile de calculer leur similarité cosinus :

In [ ]:
cosine = cosine_similarity(vectors, vectors)
np.fill_diagonal(cosine, 0)

pd.DataFrame(
    data=cosine,
    columns=vectorizer.get_feature_names_out(),
    index=vectorizer.get_feature_names_out()
)

Identifier le terme le plus similaire à un autre revient à repérer l’indice du vecteur dont le score est le plus haut :

In [ ]:
# vector cat
cat = cosine[1]
# which vector has the highest score?
most_similar = np.argmax(cat)

# which vector is it?
df_unigrams.index[most_similar]

Dans la réalité, plutôt que d’analyser l’information mutuelle ponctuelle d’un terme et de son contexte direct droit ou gauche, il est bien plus fréquent de considérer des *n*-grammes avec une fenêtre de tolérance. Par exemple, dans la phrase :

```text
Le petit chat boit du lait.
```

Le bigramme (*chat*, *lait*) peut être retenu si l’on accepte un contexte avec une fenêtre de tolérance de 3 mots. Il constitue alors ce que l’on appelle un *skip-gram* de degré 2 et de distance 3.

La librairie *NLTK* fournit un outil pour les recenser à partir de la liste ordonnée des mots d’une phrase :

In [ ]:
from nltk.util import skipgrams

s = "Le petit chat boit du lait".split()
list(skipgrams(s, n=2, k=3))